In [1]:
from dotenv import load_dotenv
import os
from array import array
from PIL import Image, ImageDraw
import sys
import time
from matplotlib import pyplot as plt
import numpy as np
from pprint import pprint

# import namespaces
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

In [2]:
load_dotenv()
cog_endpoint = os.getenv('COG_SERVICE_ENDPOINT')
cog_key = os.getenv('COG_SERVICE_KEY')

In [3]:
image_file = 'images/building.jpg'

In [4]:
credential = CognitiveServicesCredentials(cog_key) 
cv_client = ComputerVisionClient(cog_endpoint, credential)

In [5]:
def AnalyzeImage(image_file):
    print('Analyzing', image_file)

    # Specify features to be retrieved
    features = [VisualFeatureTypes.description,
                VisualFeatureTypes.tags,
                VisualFeatureTypes.categories,
                VisualFeatureTypes.brands,
                VisualFeatureTypes.objects,
                VisualFeatureTypes.adult]
    
    
    # Get image analysis
    with open(image_file, mode="rb") as image_data:
        analysis = cv_client.analyze_image_in_stream(image_data , features)

    return analysis

    # # Get image description
    # for caption in analysis.description.captions:
    #     print("Description: '{}' (confidence: {:.2f}%)".format(caption.text, caption.confidence * 100))

    # # Get image tags
    # if (len(analysis.tags) > 0):
    #     print("Tags: ")
    #     for tag in analysis.tags:
    #         print(" -'{}' (confidence: {:.2f}%)".format(tag.name, tag.confidence * 100))


    # # Get image categories
    # if (len(analysis.categories) > 0):
    #     print("Categories:")
    #     landmarks = []
    #     for category in analysis.categories:
    #         # Print the category
    #         print(" -'{}' (confidence: {:.2f}%)".format(category.name, category.score * 100))
    #         if category.detail:
    #             # Get landmarks in this category
    #             if category.detail.landmarks:
    #                 for landmark in category.detail.landmarks:
    #                     if landmark not in landmarks:
    #                         landmarks.append(landmark)

    #     # If there were landmarks, list them
    #     if len(landmarks) > 0:
    #         print("Landmarks:")
    #         for landmark in landmarks:
    #             print(" -'{}' (confidence: {:.2f}%)".format(landmark.name, landmark.confidence * 100))



    # # Get brands in the image
    # if (len(analysis.brands) > 0):
    #     print("Brands: ")
    #     for brand in analysis.brands:
    #         print(" -'{}' (confidence: {:.2f}%)".format(brand.name, brand.confidence * 100))


    # # Get objects in the image
    # if len(analysis.objects) > 0:
    #     print("Objects in image:")

    #     # Prepare image for drawing
    #     fig = plt.figure(figsize=(8, 8))
    #     plt.axis('off')
    #     image = Image.open(image_file)
    #     draw = ImageDraw.Draw(image)
    #     color = 'cyan'
    #     for detected_object in analysis.objects:
    #         # Print object name
    #         print(" -{} (confidence: {:.2f}%)".format(detected_object.object_property, detected_object.confidence * 100))
            
    #         # Draw object bounding box
    #         r = detected_object.rectangle
    #         bounding_box = ((r.x, r.y), (r.x + r.w, r.y + r.h))
    #         draw.rectangle(bounding_box, outline=color, width=3)
    #         plt.annotate(detected_object.object_property,(r.x, r.y), backgroundcolor=color)
    #     # Save annotated image
    #     plt.imshow(image)
    #     outputfile = 'objects.jpg'
    #     fig.savefig(outputfile)
    #     print('  Results saved in', outputfile)


    # # Get moderation ratings
    # ratings = 'Ratings:\n -Adult: {}\n -Racy: {}\n -Gore: {}'.format(analysis.adult.is_adult_content,
    #                                                                     analysis.adult.is_racy_content,
    #                                                                     analysis.adult.is_gory_content)
    # print(ratings)

def GetThumbnail(image_file):
    print('Generating thumbnail')

    # Generate a thumbnail
    with open(image_file, mode="rb") as image_data:
        # Get thumbnail data
        thumbnail_stream = cv_client.generate_thumbnail_in_stream(100, 100, image_data, True)

    # Save thumbnail image
    thumbnail_file_name = 'thumbnail.png'
    with open(thumbnail_file_name, "wb") as thumbnail_file:
        for chunk in thumbnail_stream:
            thumbnail_file.write(chunk)

    print('Thumbnail saved in.', thumbnail_file_name)

In [6]:
result = AnalyzeImage(image_file)

Analyzing images/building.jpg


In [10]:
vars(result)

{'additional_properties': {},
 'categories': [<azure.cognitiveservices.vision.computervision.models._models_py3.Category at 0x1f66246acb0>,
 'adult': <azure.cognitiveservices.vision.computervision.models._models_py3.AdultInfo at 0x1f651d1ba00>,
 'color': None,
 'image_type': None,
 'tags': [<azure.cognitiveservices.vision.computervision.models._models_py3.ImageTag at 0x1f66246a6b0>,
 'description': <azure.cognitiveservices.vision.computervision.models._models_py3.ImageDescriptionDetails at 0x1f66246ab60>,
 'faces': None,
 'objects': [<azure.cognitiveservices.vision.computervision.models._models_py3.DetectedObject at 0x1f662468670>],
 'brands': [],
 'request_id': '9dd7462c-a81e-42ad-9989-451c036ad8d8',
 'metadata': <azure.cognitiveservices.vision.computervision.models._models_py3.ImageMetadata at 0x1f66246abf0>}

In [16]:
vars(result.objects[0].rectangle)

{'additional_properties': {}, 'x': 183, 'y': 43, 'w': 535, 'h': 369}

In [9]:
for tag in result.tags:
    print(tag)


{'additional_properties': {}, 'name': 'outdoor', 'confidence': 0.9998234510421753, 'hint': None}
{'additional_properties': {}, 'name': 'road', 'confidence': 0.9998223781585693, 'hint': None}
{'additional_properties': {}, 'name': 'building', 'confidence': 0.9992225170135498, 'hint': None}
{'additional_properties': {}, 'name': 'street', 'confidence': 0.9953342080116272, 'hint': None}
{'additional_properties': {}, 'name': 'land vehicle', 'confidence': 0.9367624521255493, 'hint': None}
{'additional_properties': {}, 'name': 'way', 'confidence': 0.9366739988327026, 'hint': None}
{'additional_properties': {}, 'name': 'dog', 'confidence': 0.9355420470237732, 'hint': None}
{'additional_properties': {}, 'name': 'vehicle', 'confidence': 0.9255756139755249, 'hint': None}
{'additional_properties': {}, 'name': 'car', 'confidence': 0.9137619733810425, 'hint': None}
{'additional_properties': {}, 'name': 'sidewalk', 'confidence': 0.8751448392868042, 'hint': None}
{'additional_properties': {}, 'name': '

In [19]:
for category in result.categories:
    print(category)

{'additional_properties': {}, 'name': 'outdoor_', 'score': 0.00390625, 'detail': <azure.cognitiveservices.vision.computervision.models._models_py3.CategoryDetail object at 0x0000025BDC53B160>}
{'additional_properties': {}, 'name': 'outdoor_road', 'score': 0.55859375, 'detail': <azure.cognitiveservices.vision.computervision.models._models_py3.CategoryDetail object at 0x0000025BDC53B9A0>}


In [11]:
for caption in result.description.captions:
    print(caption)

{'additional_properties': {}, 'text': 'a person walking a dog on a sidewalk', 'confidence': 0.4695983827114105}


In [15]:
vars(result.adult)

{'additional_properties': {},
 'is_adult_content': False,
 'is_racy_content': False,
 'is_gory_content': False,
 'adult_score': 0.00780514208599925,
 'racy_score': 0.013629540801048279,
 'gore_score': 0.010736235417425632}

In [25]:
for object in result.objects:
    print(object)

{'additional_properties': {}, 'rectangle': <azure.cognitiveservices.vision.computervision.models._models_py3.BoundingRect object at 0x0000025BCBE70100>, 'object_property': 'car', 'confidence': 0.724, 'parent': <azure.cognitiveservices.vision.computervision.models._models_py3.ObjectHierarchy object at 0x0000025BCBE72950>}
{'additional_properties': {}, 'rectangle': <azure.cognitiveservices.vision.computervision.models._models_py3.BoundingRect object at 0x0000025BCBE71E10>, 'object_property': 'taxi', 'confidence': 0.77, 'parent': <azure.cognitiveservices.vision.computervision.models._models_py3.ObjectHierarchy object at 0x0000025BCBE726E0>}
{'additional_properties': {}, 'rectangle': <azure.cognitiveservices.vision.computervision.models._models_py3.BoundingRect object at 0x0000025BCBE723B0>, 'object_property': 'person', 'confidence': 0.781, 'parent': None}
{'additional_properties': {}, 'rectangle': <azure.cognitiveservices.vision.computervision.models._models_py3.BoundingRect object at 0x0

In [27]:
print(result.objects[0].rectangle)

{'additional_properties': {}, 'x': 0, 'y': 212, 'w': 282, 'h': 149}
